In [1]:
import os
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch

In [2]:
df = pd.read_csv('/kaggle/input/luna16-2dslices/nodule_metadata.csv')
df.head()

,seriesuid,ann_idx,z,y,x,filename,label
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,0,266,328,173,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1,252,293,220,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2,230,251,379,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,3,142,336,377,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028...,0,76,289,354,1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028...,1


In [3]:
## we filter thoe that are not nodules

df = df[df['label'] == 1]
df['label'].value_counts()

label
1    606
Name: count, dtype: int64

In [4]:
len(set(df['seriesuid'])), len(df['seriesuid'])

(311, 606)

In [5]:
class LunaDataset(Dataset):
    
    def __init__(self, dataframe, data_path, transforms=None):
        self.df = dataframe
        self.data_path = data_path
        self.transforms = transforms

        self.X = []
        self.y = []
        self.__load_dataset()

    def __load_file_images(self, filename):
        blob = np.load(filename)
        img = blob['arr_0']
        img = np.stack([img, img, img])
        self.X.append(img)

    
    def __load_dataset(self):
        self.df['filename'].apply(lambda f: self.__load_file_images(os.path.join(self.data_path, f)))
        cx = self.df['x']
        cy = self.df['y']

        self.X = np.array(self.X)
        self.y = np.stack([cx, cy], axis=-1)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, i):

        img, label = self.X[i], self.y[i]

        img = np.array(img).astype(np.float32).transpose(1, 2, 0)
        label = label.astype(np.float32)
        
        min_v = img.min()
        max_v = img.max()

        img = (img - min_v)/(max_v - min_v)
        
        if self.transforms:

            transformed = self.transforms(image=img, keypoints=[tuple(label)])
            img = transformed['image']
            label = torch.tensor(transformed['keypoints'][0], dtype=torch.float32)
            
        img = torch.tensor(img).permute(2, 0, 1)
        
        return img, label

In [6]:
patients = list(set(df['seriesuid']))
l = len(patients)
t = int(0.8*l)
train_patients = patients[:t]
val_patients = patients[t:]

In [7]:
len(train_patients), len(val_patients), len(train_patients) + len(val_patients)

(248, 63, 311)

In [8]:
train_df = df[df['seriesuid'].isin(train_patients)]
val_df = df[df['seriesuid'].isin(val_patients)]

In [9]:
from torchvision import transforms
import albumentations as A

train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,)),  # for grayscale
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))


val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.5], std=[0.5])
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

train_dataset = LunaDataset(train_df, '/kaggle/input/luna16-2dslices/dataset', train_transform)
val_dataset = LunaDataset(val_df, '/kaggle/input/luna16-2dslices/dataset', val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [10]:
img, label = train_dataset[0]
img.shape, label

(torch.Size([3, 224, 224]), tensor([ 78.4641, 129.8737]))

In [11]:
from torchvision import models
import torch.nn as nn

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 🔧 Load torchvision ViT
model = models.vit_b_16(pretrained=True)
# model = models.resnet50(pretrained=True)
model.heads.head = nn.Sequential(
    nn.Linear(model.heads.head.in_features, 128),
    nn.ReLU(),
    nn.Linear(128, 2)
)  # Output: x, y coordinates
model.to(DEVICE)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:03<00:00, 93.7MB/s] 


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [ ]:
from tqdm import tqdm

EPOCHS = 300
LR = 1e-3

criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=15)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Training"):
        images = images.to(DEVICE)
        labels = labels.float().to(DEVICE)  # shape: (B, 2)

        images = images.unsqueeze(1) if images.ndim == 3 else images

        optimizer.zero_grad()
        outputs = model(images)  # shape: (B, 2)
        # print(outputs, labels)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)

    avg_train_loss = total_loss / len(train_loader.dataset)
    print(f"Train Loss: {avg_train_loss:.4f}")

    # 📉 Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Validation"):
            images = images.to(DEVICE)
            labels = labels.float().to(DEVICE)

            images = images.unsqueeze(1) if images.ndim == 3 else images
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

    avg_val_loss = val_loss / len(val_loader.dataset)

    scheduler.step(avg_val_loss)
    print(f"Val Loss: {avg_val_loss:.4f}")


Epoch 1/300 - Training: 100%|██████████| 61/61 [00:15<00:00,  3.95it/s]


Train Loss: 67.4208


Epoch 1/300 - Validation: 100%|██████████| 16/16 [00:01<00:00,  9.57it/s]


Val Loss: 39.4464


Epoch 2/300 - Training: 100%|██████████| 61/61 [00:14<00:00,  4.07it/s]


...

Epoch 250/300 - Validation: 100%|██████████| 16/16 [00:01<00:00,  9.41it/s]


Val Loss: 38.9475


Epoch 251/300 - Training: 100%|██████████| 61/61 [00:14<00:00,  4.15it/s]


Train Loss: 38.4832


Epoch 251/300 - Validation:  12%|█▎        | 2/16 [00:00<00:01,  9.55it/s]

In [ ]:
from sklearn.metrics import mean_squared_error

model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE)
        images = images.unsqueeze(1) if images.ndim == 3 else images

        outputs = model(images)  # shape: (B, 2)
        
        plt.figure()
        plt.imshow(images[0].cpu().permute(1, 2, 0).mean(axis=-1) * 0.5 + 0.5, cmap='gray')
        x, y = outputs[0].cpu()
        a, b = labels[0].cpu()
        print(x, y, a, b)
        
        plt.plot(x, y, 'bo', markersize=5)
        plt.plot(a, b, 'ro', markersize=5)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(outputs.cpu().numpy())

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# MSE
mse = mean_squared_error(y_true, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

# Euclidean distance per point
distances = np.linalg.norm((y_true - y_pred), axis=1)
mean_distance = distances.mean()
print(f"Mean Euclidean Distance: {mean_distance:.2f} pixels")